In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests,os,sys
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime
import DataFrameGetter as dfg
import pandas as pd
import numpy as np

In [2]:
# Constants
CONTINENT = "Asia"
COUNTRY = "India"
COMP = "ISL"
SAVE_PATH = r"D:\DataSource"
START_SEASON = "Sept"
END_SEASON = "May"

In [3]:
ISL_MAIN_URL = "https://www.scoresway.com/en_GB/soccer/indian-super-league-2023-2024/as2gkuxuvi5zqktyqlc2t4kk4/results"
# ISL_MAIN_URL = "https://www.scoresway.com/en_GB/soccer/indian-super-league-2024-2025/2h95ink16qa9eabhpeydbyz9w/results"

In [4]:
def getSoup(url) :
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    sleep(3)
    return soup

def getDriver(url) :
    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)
    sleep(5)
    return driver  
    
def removeCookieDialog(driver):
    driver.execute_script("var obj = document.getElementById('onetrust-banner-sdk');if(obj){obj.remove()};")

def getMaxElement() :
    driver = getDriver(ISL_MAIN_URL)
    removeCookieDialog(driver)
    clicker = driver.find_element(By.CSS_SELECTOR, '.Opta-Exp')
    clicker.click()

    ul_element = driver.find_element(By.CSS_SELECTOR, 'ul.Opta-Cf')
    max_index = len(ul_element.find_elements(By.TAG_NAME, 'li'))
    driver.close()
    return max_index

In [5]:

def get_all_matches_links(season_url) :
    mw_result_links = {}
    backup_of_urls = []
    max_index = getMaxElement()
    
    for i in range(max_index) :
        inner_driver = getDriver(ISL_MAIN_URL)
        removeCookieDialog(inner_driver)
        clicker = inner_driver.find_element(By.CSS_SELECTOR, '.Opta-Exp')
        clicker.click()
        ul_element = inner_driver.find_element(By.CSS_SELECTOR, 'ul.Opta-Cf')
        
        all_mw = ul_element.find_elements(By.TAG_NAME, 'a')
        if len(all_mw) < i :
            print(i,"Overflow",all_mw)
        mw_item  = all_mw[i]
        name = mw_item.text
        print(i,"WORKING FOR : ",name,mw_item.tag_name)
        # continue
        mw_item.click()
        fixture_tables =[x for x in inner_driver.find_elements(By.CSS_SELECTOR,'tbody.Opta-result.Opta-fixture') if x.text]
        urls = []
        for j,fixture in enumerate(fixture_tables) :
            try :
                print(f"Working on {name}, Game {j+1}")
                match_page = fixture.find_element(By.CSS_SELECTOR, '[title="View match"]')
                match_page.click()
                # sleep(5)
                removeCookieDialog(inner_driver)
                # results_panes = inner_driver.find_elements(By.CSS_SELECTOR,'ul.striplist')
                result_urls = [x.find_element(By.TAG_NAME,'a').get_property('href')
                         for x in inner_driver.find_element(By.CSS_SELECTOR,'ul.striplist').find_elements(By.TAG_NAME,'li') 
                         if x.text == 'PLAYER STATS']
                if result_urls :
                    urls.append((j,result_urls[0]))
                inner_driver.execute_script("window.history.go(-1)")
                # break
            except Exception as e :
                print(type(e),print(e))
                break
        backup_of_urls.append(urls)
        inner_driver.close()
    return backup_of_urls

In [6]:
backup_of_urls = get_all_matches_links(ISL_MAIN_URL)
all_urls = []
n = [all_urls.extend(x) for x in backup_of_urls]

0 WORKING FOR :   a
Working on , Game 1
Working on , Game 2
Working on , Game 3
Working on , Game 4
Working on , Game 5
Working on , Game 6
Working on , Game 7
1 WORKING FOR :   a
Working on , Game 1
Working on , Game 2
Working on , Game 3
Working on , Game 4
Working on , Game 5
Working on , Game 6
2 WORKING FOR :   a
Working on , Game 1
Working on , Game 2
Working on , Game 3
Working on , Game 4
Working on , Game 5
Working on , Game 6
Working on , Game 7
Working on , Game 8
3 WORKING FOR :   a
Working on , Game 1
Working on , Game 2
Working on , Game 3
Working on , Game 4
Working on , Game 5
Working on , Game 6
4 WORKING FOR :   a
Working on , Game 1
Working on , Game 2
Working on , Game 3
Working on , Game 4
Working on , Game 5
Working on , Game 6
5 WORKING FOR :   a
Working on , Game 1
Working on , Game 2
Working on , Game 3
Working on , Game 4
Working on , Game 5
Working on , Game 6
6 WORKING FOR :   a
Working on , Game 1
Working on , Game 2
Working on , Game 3
Working on , Game 4


In [7]:
all_urls[:4]

[(0,
  'https://www.scoresway.com/en_GB/soccer/indian-super-league-2023-2024/as2gkuxuvi5zqktyqlc2t4kk4/match/view/5vqm3epot9ch15e2ldqjzypsk/player-stats'),
 (1,
  'https://www.scoresway.com/en_GB/soccer/indian-super-league-2023-2024/as2gkuxuvi5zqktyqlc2t4kk4/match/view/bilwpjmbdd439zazyty3m3pjo/player-stats'),
 (2,
  'https://www.scoresway.com/en_GB/soccer/indian-super-league-2023-2024/as2gkuxuvi5zqktyqlc2t4kk4/match/view/bhvg8hlu4jekusz8n3z6u06c4/player-stats'),
 (3,
  'https://www.scoresway.com/en_GB/soccer/indian-super-league-2023-2024/as2gkuxuvi5zqktyqlc2t4kk4/match/view/37ghgf4qkmtec8h6sr1hkbxg4/player-stats')]

In [8]:
all_urls = [y for x in backup_of_urls for y in x]

In [9]:
def get_game_events_results(url):
    driver = getDriver(url)
    removeCookieDialog(driver)
    fixture_tables =[x for x in driver.find_elements(By.CSS_SELECTOR,'tbody.Opta-result.Opta-fixture') if x.text]
    urls = []
    
    for i,fixture in enumerate(fixture_tables) :
        try :
            match_page = fixture.find_element(By.CSS_SELECTOR, '[title="View match"]')
            match_page.click()
            sleep(5)
            removeCookieDialog(driver)
            # results_panes = driver.find_elements(By.CSS_SELECTOR,'ul.striplist')
            result_urls = [x.find_element(By.TAG_NAME,'a').get_property('href')
                     for x in driver.find_element(By.CSS_SELECTOR,'ul.striplist').find_elements(By.TAG_NAME,'li') 
                     if x.text == 'PLAYER STATS']
            if result_urls :
                urls.append((i,result_urls[0]))
            driver.execute_script("window.history.go(-1)")
        except Exception as e :
            driver.close()
            break
    return urls

In [17]:
# import json
# with open("All_Links_ISL_23_24.json","w") as f :
#     f.write(json.dumps([x[1] for x in all_urls]))

In [18]:
def get_season_name(date_str):
    month_map = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
        "Jul": 7, "Aug": 8, "Sept": 9, "Oct": 10, "Nov": 11, "Dec": 12
    }
    date = datetime.strptime(date_str, "%Y-%m-%d")
    year = date.year
    month = date.month
    start_month = month_map[START_SEASON]
    
    if month >= start_month:
        season_start = year
        season_end = year + 1
    else:
        season_start = year - 1
        season_end = year

    return f"{str(season_start)[-2:]}-{str(season_end)[-2:]}"


In [19]:
def create_needed_directories(continent,country,comp,season) :
    cont_path = os.path.join(SAVE_PATH, continent)
    os.makedirs(cont_path, exist_ok=True)

    country_path = os.path.join(SAVE_PATH, continent, country)
    os.makedirs(country_path, exist_ok=True)

    comp_path = os.path.join(SAVE_PATH, continent, country, comp)
    os.makedirs(comp_path, exist_ok=True)

    season_path = os.path.join(SAVE_PATH, continent, country, comp, season)
    os.makedirs(season_path, exist_ok=True)
    return season_path


In [20]:
f_U_ = []
for i,url in all_urls[1:]:
    try:
        response = dfg.fetch_event_response(url)
        match_info = response['matchevent']['response_content']['matchInfo']
        home = [x['officialName'].replace(' ','_') for x in match_info['contestant'] if x.get('position') == 'home'][0]
        away = [x['officialName'].replace(' ','_') for x in match_info['contestant'] if x.get('position') == 'away'][0]
        file_base = create_needed_directories(CONTINENT,COUNTRY,COMP,get_season_name(match_info['localDate']))
        file_name = f"{match_info['localDate'].replace('-','_')}_{home}_vs_{away}.xlsx"
        full_file_path = os.path.join(file_base,file_name)
        if os.path.exists(full_file_path):
            print(f"Bro {file_name} toh hai already !!!")
            print()
            continue
    # if os.exis
        element = response['matchevent']
        content_element = element['response_content']
        # Get Team names
        teams_df = pd.DataFrame(content_element['matchInfo']['contestant'])
        # Get opta-event IDs
        type_id_df = pd.read_csv(r'./opta_data/template/opta-events.csv', encoding = 'ISO-8859-1')
        type_id_df.rename(columns={'ID':'typeId'},inplace=True)
        # Get opta-qualifiers IDs
        qualifiers_df = pd.read_csv(r'./opta_data/template/opta-qualifiers.csv')
        # Preparing correct format of data
        events_df = pd.DataFrame(content_element['liveData']['event'])
        # L1
        id1 = teams_df['id'].iloc[0]
        team1 = teams_df['officialName'].iloc[0]
        id2 = teams_df['id'].iloc[1]
        team2 = teams_df['officialName'].iloc[1]
        
        events_df['contestantId'] = np.where(events_df['contestantId']==id1,team1,team2)
        events_df['outcome'] = np.where(events_df['outcome']==1,'Successful','Unsuccessful')
        events_df.rename(columns={"contestantId":'Team','timeMin':'Minute','timeSec':'Seconds'},inplace=True)
        # L2        
        event_action_result = events_df.apply(dfg.return_event_data_and_description,args=(type_id_df,),axis=1)
        # Unpack the result into two new columns
        events_df['typeId'],events_df['Description'] = zip(*event_action_result)
        events_df['qualifier'] = events_df.apply(dfg.return_qualifier_data,args=(qualifiers_df,),axis=1)
        events_df.to_excel(full_file_path,index=False)
    except Exception as e:
        print(e)
        print("Fuck up for  : ",url)
        f_U_.append(url)

Bro 2024_04_29_Mumbai_City_FC_vs_FC_Goa.xlsx toh hai already !!!

Bro 2024_04_28_Mohun_Bagan_Super_Giant_vs_Odisha_FC.xlsx toh hai already !!!



KeyboardInterrupt: 